In [2]:
import pandas as pd
import mysql.connector as mysql
from mysql.connector import Error
from dotenv import load_dotenv
import os
import numpy as np

In [3]:
load_dotenv()


try:
   conn = mysql.connect(
      host=os.getenv("HOST"),
      user=os.getenv("USER"),
      password=os.getenv("PASSWORD"),
      database=os.getenv("DB_NAME")
   )
   print(f"Connection to MySQL DB successful")
except Error:
   print(Error)




Connection to MySQL DB successful


In [13]:
# Colaborador sem gerente
query1 = ''' 
SELECT CONCAT(e.Fname, ' ', e.Minit, ' ', e.Lname) AS Colaborador,
      CONCAT(s.Fname, ' ', s.Minit, ' ', s.Lname) AS Gerente,
      e.Ssn

FROM employee e

LEFT JOIN employee s ON e.Super_ssn = s.Ssn
WHERE s.Ssn IS NOT NULL;
'''

df1 = pd.read_sql_query(query1, conn)
# df1


C:\Users\felip\AppData\Local\Temp\ipykernel_1692\2646394697.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query1, conn)


In [14]:
df1

,Colaborador,Gerente,Ssn
0,John B Smith,Franklin T Wong,123456789
1,Franklin T Wong,James E Borg,333445555
2,Joyce A English,Franklin T Wong,453453453
3,Ramesh K Narayan,Franklin T Wong,666884444
4,Jennifer S Wallace,James E Borg,987654321
5,Ahmad V Jabbar,Jennifer S Wallace,987987987
6,Alicia J Zelaya,Jennifer S Wallace,999887777


In [6]:
colaborador_sem_superior = []
colaborador_sem_superior.extend(row['Colaborador']
                                for index, row in df1.iterrows()
                                if pd.isnull(row['Superior']))
print('Chefe:',end=' ')
for colaborador in colaborador_sem_superior:
   print(f'{colaborador} ',end='')
df1

Chefe: James E Borg 

,Colaborador,Superior
0,John B Smith,Franklin T Wong
1,Franklin T Wong,James E Borg
2,Joyce A English,Franklin T Wong
3,Ramesh K Narayan,Franklin T Wong
4,James E Borg,NaN
5,Jennifer S Wallace,James E Borg
6,Ahmad V Jabbar,Jennifer S Wallace
7,Alicia J Zelaya,Jennifer S Wallace


In [15]:
df1.to_excel('../Desafio de Projeto/queries_results/colaboradores_sem_supervisor.xlsx', index=False)

In [6]:
# Departamentos e gerentes

query2 = ''' 
   SELECT d.Dname AS Departamento,
      CONCAT(e.Fname, ' ', e.Minit, ' ', e.Lname) AS Gerente,
      e.Ssn
   FROM departament d
   JOIN employee e ON e.Ssn = d.Mgr_ssn;
'''
df2 = pd.read_sql_query(query2, conn)
df2


C:\Users\felip\AppData\Local\Temp\ipykernel_1692\880120119.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query2, conn)


,Departamento,Gerente,Ssn
0,Headquarters,James E Borg,888665555
1,Administration,Jennifer S Wallace,987654321
2,Research,Franklin T Wong,333445555


In [12]:
df2.to_excel('../Desafio de Projeto/queries_results/departamentos_e_gerentes.xlsx', index=False)

In [4]:
# Horas por projeto
query3 = ''' 
   SELECT
      CONCAT(e.Fname, ' ', e.Minit, ' ', e.Lname) AS Colaborador,
      p.Pname AS Projeto,
      d.Dname AS Departamento,
      wo.Hours AS Horas
   FROM works_on wo
   JOIN employee e ON wo.Essn = e.Ssn
   JOIN project p ON wo.Pno = p.Pnumber
   JOIN departament d ON p.Dnum = d.Dnumber;


'''

df3 = pd.read_sql_query(query3, conn)
df3


C:\Users\felip\AppData\Local\Temp\ipykernel_1692\2100763029.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3 = pd.read_sql_query(query3, conn)


,Colaborador,Projeto,Departamento,Horas
0,John B Smith,ProductX,Research,32.5
1,Joyce A English,ProductX,Research,20.0
2,John B Smith,ProductY,Research,7.5
3,Franklin T Wong,ProductY,Research,10.0
4,Joyce A English,ProductY,Research,20.0
5,Franklin T Wong,ProductZ,Research,10.0
6,Ramesh K Narayan,ProductZ,Research,40.0
7,Franklin T Wong,Computerization,Administration,10.0
8,Ahmad V Jabbar,Computerization,Administration,35.0
9,Alicia J Zelaya,Computerization,Administration,10.0


In [5]:
df3.to_excel('../Desafio de Projeto/queries_results/horas_por_projeto.xlsx', index=False)

In [ ]:
# Separando colunas complexas
